<a id='Read-the-results-of-the-dictionary-after-GPU-computation'></a>
# Read the Results of the Dictionary After GPU Computation


## Library Dependencies:

 - **pip install pillow**
 - **pip install seaborn**
 - also requires these libraries if they are not already installed: **matplotlib, numpy, pandas, sklearn**

In [ ]:
from lab.Read_Transform_Images import *
# read results from json file in results folder
resultsDict = read_results_json()
# get list_PIL_Images from Read_Tansform_Images
resultsDict = Read_Transform_Images(resultsDict)

<a id='Plot-the-histogram-of-the-kmeans-clusters-using-GPU-results'></a>
# Plot Kmeans Clusters 

Plot histogram of the using GPU results
- _Code:_ [Back to Sections](#Sections)

In [ ]:
#resultsDict = Compute_kmeans_db_histogram_labels(resultsDict, knee = 6, gpu_available = gpu_available) #knee = 5
counts = np.asarray(resultsDict['counts'])
bins = np.asarray(resultsDict['bins'])
plt.xlabel("Weight")
plt.ylabel("Probability")
plt.title("Histogram with Probability Plot")
slice = min(counts.shape[0], bins.shape[0])
plt.bar(bins[:slice],counts[:slice])
plt.grid()
plt.show()

In [ ]:
print(resultsDict['bins'])
print(resultsDict['counts'])
resultsDict.keys()

# Display Similar Images

Visually compare image which have been clustered by the allgorithm

In [ ]:
clusterRank = 1
d = {i:cts for i, cts in enumerate(resultsDict['counts'])}
sorted_d = sorted(d.items(), key=operator.itemgetter(1), reverse=True)

id = sorted_d[clusterRank][0]
indexCluster = np.where(np.asarray(resultsDict['km_labels']) == id  )[0].tolist()
img_arr = []
for idx in indexCluster:
    img_arr.append(np.array((resultsDict['list_PIL_Images'][idx])))
img_arr = np.asarray(img_arr)
displayImageGrid(img_arr)

# PlotPlot Seaborn KMEANS Clusters

Indicates numbers of images that are close in color space

In [ ]:
%matplotlib inline

n_components = 4

columns = ['PC{:0d}'.format(c) for c in range(n_components)]
data = pd.DataFrame(np.asarray(resultsDict['PCA_fit_transform'])[:,:n_components], columns = columns)
#k_means = resultsDict['model']
data['cluster'] = resultsDict['km_labels']
data.head()
# similarlyNamedImages = [9,6,6,8,6,4,8,3]
# print('number of similarly named images: ', similarlyNamedImages)

columns.append('cluster')
plt.figure(figsize=(8,8))

sns.set_context('notebook');
g = sns.pairplot(data[columns], hue="cluster", palette="Paired", diag_kws=dict(hue=None));
g.fig.suptitle("KMEANS pairplot");

# Find DBSCAN EPS parameter

Density-Based Spatial Clustering of Applications with Noise (DBSCAN) finds core samples of high density and expands clusters from them. Good for data which contains clusters of similar density.

EPS: "epsilon" value in sklearn is the maximum distance between two samples for one to be considered as in the neighborhood of the other.

At least a first value to start. We are using kNN to find distances commonly occuring in the dataset. Values of EPS below this threshold distance will be considered as lyig within a given cluster. This means we should look for long flat plateaus and read the y coordinate off the kNN plot to get a starting value for EPS.

Different datasets can have wildly different sweet spots for EPS. Some datasets require EPS values of .001 other datasets may work best with values of EPS of several thousand. We use this trick to get in the right or approximate neighborhood of the EPS.


In [ ]:
from sklearn.neighbors import NearestNeighbors

PCA_images = resultsDict['PCA_fit_transform']

neighbors = NearestNeighbors(n_neighbors=2)
#X = StandardScaler().fit_transform(PCA_images)
neighbors_fit = neighbors.fit(PCA_images)
distances, indices = neighbors_fit.kneighbors(PCA_images)

distances = np.sort(distances, axis=0)
plt.xlabel('number of images')
plt.ylabel('distances')
plt.title('KNN Distances Plot')
plt.plot(distances[:,1])
plt.grid()

# Use DBSCAN to find clusters

we will use initial estiamtes from KNN above (find elbow) to given initial trial for DBSCAN EPS values 

In the plot above, there is a plateau in the y values somewherre near 350 indicating that a cluster distance (aka EPS) might work well somewhere near this value. We used this value in the batch_clustering_Streamlined.py file when computing DBSCAN.

**EPS:** Two points are  neighbors if the distance between the two points is below a threshold.
**n:** The minimum number of neighbors a given point should have in order to be classified as a core point. 
The point itself is included in the minimum number of samples.

# Below: Sort DBSCAN Results by Cluster Size

In [ ]:
#%%write_and_run    lab/compute_DBSCANClusterRank.py 
def compute_DBSCANClusterRank(n, EPS):
    d = {index-1:int(cnt) for index, cnt in enumerate(counts )}
    sorted_d = sorted(d.items(), key=operator.itemgetter(1), reverse=True)
    for i in range(0, len(d)):
        idx = sorted_d[i][0]
        print('cluster = ', idx, ' occurs', int(sorted_d[i][1]), ' times')
    return db, counts, bins, sorted_d

n_components = 4

columns = ['PC{:0d}'.format(c) for c in range(n_components)]
data = pd.DataFrame(np.asarray(resultsDict['PCA_fit_transform'])[:,:n_components], columns = columns)

columns.append('cluster')
data['cluster'] = resultsDict['db_labels'] 
data.head()

<a id='Plot-DBSCAN'></a>
# DBSCAN Cluster Plot

Plot a histogram of the using GPU results
- _Code:_ [Back to Sections](#Sections)

 

To indicate numbers of images in each cluster. color each point by its membership in a cluster

In [ ]:
%matplotlib inline

sns.set_context('notebook');
g = sns.pairplot(data[columns], hue="cluster", palette="Paired", diag_kws=dict(hue=None));
g.fig.suptitle("DBSCAN pairplot");

In [ ]:
data.columns

# Print Filenames of Outliers

In [ ]:
print('Outlier/problematic images are: \n', 
      [resultsDict['imagesFilenameList'][f] for f in list(data[data['cluster'] == -1].index)]
     )

# Final Thoughts and next steps...

You may have noticed how difficult it is to get decent clustering on even 30 or 40 images using only RGB or HSV as the feature set. 

If all the images are well separated in either RGB, or HSV color space then these features are useful for clustering 

However, a suggested next step - or next training - would be to encode the data differently.  Perhaps using image classification with VGG16 but removing the last layer as a pre-process prior to k-means or DBSCAN


# Notices & Disclaimers 

Intel technologies may require enabled hardware, software or service activation.
No product or component can be absolutely secure.

Your costs and results may vary.

© Intel Corporation. Intel, the Intel logo, and other Intel marks are trademarks of Intel Corporation or its subsidiaries. 
*Other names and brands may be claimed as the property of others.

